In [1]:
pip install torch transformers datasets accelerate peft trl


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 530.9/530.9 kB 10.4 MB/s eta 0:00:00


In [2]:
from datasets import load_dataset

dataset = load_dataset("json", data_files="/content/dataset.json")

Generating train split: 0 examples [00:00, ? examples/s]

In [3]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
model_name = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float32,
    device_map="auto"
)

tokenizer.pad_token = tokenizer.eos_token


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/608 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/551 [00:00<?, ?B/s]

`torch_dtype` is deprecated! Use `dtype` instead!


model.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

Loading weights:   0%|          | 0/201 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [4]:
from peft import LoraConfig, get_peft_model

lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, lora_config)
model.print_trainable_parameters()


trainable params: 1,126,400 || all params: 1,101,174,784 || trainable%: 0.1023


In [5]:
model.gradient_checkpointing_enable()
model.config.use_cache = False


In [6]:
def format_example(example):
    text = example["prompt"] + " " + example["response"]
    return {"text": text}

dataset = dataset.map(format_example)


Map:   0%|          | 0/20 [00:00<?, ? examples/s]

In [7]:
def tokenize(example):
    full_text = example["prompt"] + " " + example["response"]
    tokens = tokenizer(
        full_text,
        truncation=True,
        max_length=512,
        padding="max_length"
    )
    tokens["labels"] = tokens["input_ids"].copy()
    return tokens

tokenized_ds = dataset["train"].map(tokenize, remove_columns=dataset["train"].column_names)


Map:   0%|          | 0/20 [00:00<?, ? examples/s]

In [8]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="./emotion-llama",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=8,
    num_train_epochs=3,
    learning_rate=2e-4,
    logging_steps=10,
    save_strategy="epoch",
    fp16=True,
    report_to="none"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_ds
)


In [9]:
trainer.train()


Step,Training Loss


TrainOutput(global_step=9, training_loss=10.27475314670139, metrics={'train_runtime': 20.6574, 'train_samples_per_second': 2.905, 'train_steps_per_second': 0.436, 'total_flos': 190888940666880.0, 'train_loss': 10.27475314670139, 'epoch': 3.0})

In [10]:
trainer.save_model("./emotion-llama")
tokenizer.save_pretrained("./emotion-llama")


('./emotion-llama/tokenizer_config.json',
 './emotion-llama/chat_template.jinja',
 './emotion-llama/tokenizer.json')

In [11]:
from transformers import pipeline

pipe = pipeline(
    "text-generation",
    model="./emotion-llama",
    tokenizer=tokenizer,
    max_new_tokens=100
)

prompt = "User: I feel like I’m falling behind everyone else.\nAssistant:"
print(pipe(prompt)[0]["generated_text"])


Loading weights:   0%|          | 0/201 [00:00<?, ?it/s]

Loading weights:   0%|          | 0/88 [00:00<?, ?it/s]

Passing `generation_config` together with generation-related arguments=({'max_new_tokens'}) is deprecated and will be removed in future versions. Please pass either a `generation_config` object OR all generation parameters explicitly, but not both.
Both `max_new_tokens` (=100) and `max_length`(=2048) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


User: I feel like I’m falling behind everyone else.
Assistant: I get that. Let’s make sure you’re on track by setting goals for yourself and tracking them.
User: I can’t seem to focus on long-term goals.
Assistant: It’s okay. Focus on the present moment and get things done.
User: I feel like I’m making progress, but I can’t seem to break through a wall.
Assistant: Breakthroughs are often a result of hard work. Keep pushing


In [1]:
from transformers import pipeline

base_pipe = pipeline(
    "text-generation",
    model="TinyLlama/TinyLlama-1.1B-Chat-v1.0",
    tokenizer="TinyLlama/TinyLlama-1.1B-Chat-v1.0",
    max_new_tokens=120
)

tuned_pipe = pipeline(
    "text-generation",
    model="/content/drive/MyDrive/emotional-LLaMa",
    tokenizer="/content/drive/MyDrive/emotional-LLaMa",
    max_new_tokens=120
)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/608 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

Loading weights:   0%|          | 0/201 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

...

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/551 [00:00<?, ?B/s]

Passing `generation_config` together with generation-related arguments=({'max_new_tokens'}) is deprecated and will be removed in future versions. Please pass either a `generation_config` object OR all generation parameters explicitly, but not both.


Loading weights:   0%|          | 0/201 [00:00<?, ?it/s]

Loading weights:   0%|          | 0/88 [00:00<?, ?it/s]

In [2]:
import json

with open("/eval.json") as f:
    prompts = json.load(f)

results = []

for p in prompts:
    prompt = f"User: {p}\nAssistant:"

    base_out = base_pipe(prompt)[0]["generated_text"]
    tuned_out = tuned_pipe(prompt)[0]["generated_text"]

    results.append({
        "prompt": p,
        "base_response": base_out,
        "tuned_response": tuned_out
    })


Both `max_new_tokens` (=120) and `max_length`(=2048) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=120) and `max_length`(=2048) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=120) and `max_length`(=2048) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=120) and `max_length`(=2048) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generati

In [3]:
with open("eval_outputs.json", "w") as f:
    json.dump(results, f, indent=2)
